<a href="https://colab.research.google.com/github/ChaitaliV/Randomized-Occam-s-Razor/blob/main/MuRIL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from IPython.display import Image

In [ ]:
!pip install pytorch-pretrained-bert pytorch-nlp

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 KB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.1/90.1 KB 11.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.1/135.1 KB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.5/10.5 MB 86.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.6/79.6 KB 9.5 MB/s eta 0:00:00


In [ ]:
!pip install -q transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.7/6.7 MB 89.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.2/199.2 KB 22.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 64.9 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from keras_preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer, BertConfig
from transformers import AdamW, BertForSequenceClassification, get_linear_schedule_with_warmup
from tqdm import tqdm, trange
import pandas as pd
import io
import re
import numpy as np
import matplotlib.pyplot as plt

ModuleNotFoundError: ignored

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [ ]:
model=torch.load(r'/content/drive/MyDrive/MuRIL_HASOC.pt',map_location=torch.device('cuda'))

In [ ]:
dF = pd.read_csv('/content/drive/MyDrive/translated_final_merged_all.csv')
stop_words = set(open(r'/content/drive/MyDrive/stop_words.txt').read().split())
dF.head()

,Unnamed: 0,0,1,2,3,4
0,0,0,AI_170_result_slice_140000,"I run the machine for the jeans etc., which ar...",जो जीन्स वगैरा बनती है ना जो इनकी लुपियाँ होती...,0.0
1,1,1,NaN,"Apart from this, sir, after doing normal work ...",इसके आलावा सर ( english : sir ) तो में नार्मल ...,0.0
2,2,2,NaN,"Sir, income is work in contract, Sir what happ...",सर ( english : sir ) आमदनी ठेके में काम है सर ...,0.0
3,3,3,NaN,Almost in a month - there is work according to...,एक महीने में लगभग - लगभग ठेके के हिसाब का काम ...,0.0
4,4,4,AI_170_result_slice_70000,my name is Sonu.,मेरा नाम सोनू है।,0.0


In [ ]:
def data_cleaning(df):
  """ Removing Unwanted Columns """
  df = df.drop(['1','Unnamed: 0','0','2'], axis=1)
  df.rename(columns = {'3':'text', '4' :'label'}, inplace = True)
  df = df[df['label'].notna()]
  df = df[df['text'].notna()]
  df = df.reset_index()
  df = df.drop(['index'],axis=1)
  df[['label']] = df[['label']].astype(int)

 
  """ Removing emojis, mentions, english words, characters, space, 
  websites, urls, punctuations, etc."""
  for i in range(len(df)):
   
    df['text'][i]  = re.sub("[A-Za-z0-9_]+"," ", str(df['text'][i]))    # Remove English words and Numbers
    df['text'][i]  = re.sub(r"[\'$()+=%{}~'.!:?&*-_\"\/]",r' ', str(df['text'][i]))   # Remove Punctuations
    df['text'][i]  = str(df['text'][i]).replace(' +', ' ')               # Remove more than 1 spaces
    df['text'][i]  = df['text'][i].split()
    df['text'][i]  = [word for word in df['text'][i] if not word in stop_words]
    if len(df['text'][i]) < 5:                                         # Remove strings with length less than 5
      df = df.drop(i, axis = 0)
    else: 
      df['text'][i] = " ".join(df['text'][i])

  """ Reindexing """
  df = df.reset_index()
  df = df.drop(['index'], axis = 1)
  
  return df


In [ ]:
dF['3'][314]

nan

In [ ]:
df = data_cleaning(dF)

<ipython-input-35-c349cf9fb89c>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]  = re.sub("[A-Za-z0-9_]+"," ", str(df['text'][i]))    # Remove English words and Numbers
<ipython-input-35-c349cf9fb89c>:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'][i]  = re.sub(r"[\'$()+=%{}~'.!:?&*-_\"\/]",r' ', str(df['text'][i]))   # Remove Punctuations
<ipython-input-35-c349cf9fb89c>:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-ve

In [ ]:
df

,text,label
0,जीन्स वगैरा बनती ना लुपियाँ मशीन चलता,0
1,आलावा नार्मल काम घर दोस्त यारी फन घर चले,0
2,आमदनी ठेके काम हजार काम नहीं क्योकि काम नहीं र...,0
3,महीने लगभग लगभग ठेके हिसाब काम कमाने कमा लेता ...,0
4,घर मम्मी पापा बहन भाई,0
...,...,...
3131,कहीं ना रीज़न बीमार वाइफ सोर्स तरीके इत्तेफ़ाक़ स...,2
3132,बॉस टकरा नहीं डिसिप्लिन फोर्स लिखेंगे सीनियर म...,2
3133,महीने थोड़ा दिन होंगे बस लो।,2
3134,अलग अलग सब मतलब साल बना अलग अलग चलेगा,2


In [ ]:
# Creating sentence, label lists and adding Bert tokens
sentences = df.text.values

# Adding CLS and SEP tokens at the beginning and end of each sentence for BERT
sentences = ["[CLS] " + str(sentence) + " [SEP]" for sentence in sentences]
labels = df.label.values

In [ ]:
#@title Activating the BERT Tokenizer
tokenizer = BertTokenizer.from_pretrained("google/muril-base-cased")
tokenized_texts = [tokenizer.tokenize(sent) for sent in sentences]
print ("Tokenize the first sentence:")
print(tokenized_texts[0])
print (len(tokenized_texts[0]))

Tokenize the first sentence:
['[CLS]', 'जीन्स', 'व', '##गै', '##रा', 'बनती', 'ना', 'ल', '##ुप', '##ियाँ', 'मशीन', 'चलता', '[SEP]']
13


In [ ]:
#Processing the data
# Set the maximum sequence length. The longest sequence in our training set is 47, but we'll leave room on the end anyway. 
# In the original paper, the authors used a length of 512.
MAX_LEN = 128

# Use the BERT tokenizer to convert the tokens to their index numbers in the BERT vocabulary
input_ids = [tokenizer.convert_tokens_to_ids(x) for x in tokenized_texts]
print(len(input_ids[0]))
# Pad our input tokens
input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", truncating="post", padding="post")
print(len(input_ids[0]))

13
128


In [ ]:
#Create attention masks
attention_masks = []

# Create a mask of 1s for each token followed by 0s for padding
for seq in input_ids:
  seq_mask = [float(i>0) for i in seq]
  attention_masks.append(seq_mask)

print(type(input_ids))
train_ids = input_ids[0:-500]
train_label = labels[0:-500]
test_ids = input_ids[-500:]
test_label = labels[-500:]
train_attention_masks = attention_masks[0:-500]
test_attention_masks = attention_masks[-500:]
print(len(train_label))
print(len(test_attention_masks))
print(len(train_ids))

<class 'numpy.ndarray'>
2636
500
2636


In [ ]:
# Splitting data into train and validation sets
# Use train_test_split to split our data into train and validation sets for training

train_inputs, validation_inputs, train_labels, validation_labels = train_test_split(train_ids, train_label,random_state = 2018, test_size = 0.1 )
train_masks, validation_masks, _, _ = train_test_split(train_attention_masks, train_ids,
                                             random_state=2018, test_size=0.1)

In [ ]:
type(validation_masks)

list

In [ ]:
#Converting all the data into torch tensors
# Torch tensors are the required datatype for our model

train_inputs = torch.tensor(train_inputs)
validation_inputs = torch.tensor(validation_inputs)
train_labels = torch.tensor(train_labels)
validation_labels = torch.tensor(validation_labels)
train_masks = torch.tensor(train_masks)
validation_masks = torch.tensor(validation_masks)
print(type(train_inputs))

<class 'torch.Tensor'>


In [ ]:
# Selecting a Batch Size and Creating and Iterator
# Select a batch size for training. For fine-tuning BERT on a specific task, the authors recommend a batch size of 16 or 32
batch_size = 16

# Create an iterator of our data with torch DataLoader. This helps save on memory during training because, unlike a for loop, 
# with an iterator the entire dataset does not need to be loaded into memory
print(len(train_inputs))
train_data = TensorDataset(train_inputs, train_masks, train_labels)
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)

validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
validation_sampler = SequentialSampler(validation_data)
validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

2372


In [ ]:
isinstance(model, nn.DataParallel)

True

In [ ]:
model

DataParallel(
  (module): BertForSequenceClassification(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(197285, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=Tr

In [ ]:
# Optimizer Grouped Parameters
# reference:
# https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L102

# Don't apply weight decay to any parameters whose names include these tokens.
# (Here, the BERT doesn't have `gamma` or `beta` parameters, only `bias` terms)
param_optimizer = list(model.named_parameters())
no_decay = ['bias', 'LayerNorm.weight']
# Separate the `weight` parameters from the `bias` parameters. 
# - For the `weight` parameters, this specifies a 'weight_decay_rate' of 0.01. 
# - For the `bias` parameters, the 'weight_decay_rate' is 0.0. 
optimizer_grouped_parameters = [
    # Filter for all parameters which *don't* include 'bias', 'gamma', 'beta'.
    {'params': [p for n, p in param_optimizer if not any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.1},
    
    # Filter for parameters which *do* include those.
    {'params': [p for n, p in param_optimizer if any(nd in n for nd in no_decay)],
     'weight_decay_rate': 0.0}
]
# Note - `optimizer_grouped_parameters` only includes the parameter values, not 
# the names.

In [ ]:
# The Hyperparameters for the Training Loop 

# Number of training epochs (authors recommend between 2 and 4)
epochs = 3

optimizer = AdamW(optimizer_grouped_parameters,
                  lr = 3e-5, # args.learning_rate - default is 5e-5, our notebook had 2e-5
                  eps = 1e-8 # args.adam_epsilon  - default is 1e-8.
                  )
# Total number of training steps is number of batches * number of epochs.
# `train_dataloader` contains batched data so `len(train_dataloader)` gives 
# us the number of batches.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            num_warmup_steps = 0.1, # Default value in run_glue.py
                                            num_training_steps = total_steps)

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
#Creating the Accuracy Measurement Function
# Function to calculate the accuracy of our predictions vs labels
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)

In [ ]:
# The Training Loop
t = [] 

# Store our loss and accuracy for plotting
train_loss_set = []

# trange is a tqdm wrapper around the normal python range
for _ in trange(epochs, desc="Epoch"):
  
  
  # Training
  
  # Set our model to training mode (as opposed to evaluation mode)
  model.train()
  
  # Tracking variables
  tr_loss = 0
  nb_tr_examples, nb_tr_steps = 0, 0
  
  # Train the data for one epoch
  for step, batch in enumerate(train_dataloader):
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Clear out the gradients (by default they accumulate)
    optimizer.zero_grad()
    # Forward pass
    outputs = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask, labels=b_labels)
    loss = outputs['loss']
    train_loss_set.append(loss.item())    
    # Backward pass
    loss.backward()
    # Update parameters and take a step using the computed gradient
    optimizer.step()

    # Update the learning rate.
    scheduler.step()
    
    
    # Update tracking variables
    tr_loss += loss.item()
    nb_tr_examples += b_input_ids.size(0)
    nb_tr_steps += 1

  print("Train loss: {}".format(tr_loss/nb_tr_steps))
    
    
  # Validation

  # Put model in evaluation mode to evaluate loss on the validation set
  model.eval()

  # Tracking variables 
  eval_loss, eval_accuracy = 0, 0
  nb_eval_steps, nb_eval_examples = 0, 0

  # Evaluate data for one epoch
  for batch in validation_dataloader:
    # Add batch to GPU
    batch = tuple(t.to(device) for t in batch)
    # Unpack the inputs from our dataloader
    b_input_ids, b_input_mask, b_labels = batch
    # Telling the model not to compute or store gradients, saving memory and speeding up validation
    with torch.no_grad():
      # Forward pass, calculate logit predictions
      logits = model(b_input_ids, token_type_ids=None, attention_mask=b_input_mask)
    
    # Move logits and labels to CPU
    logits = logits['logits'].detach().cpu().numpy()
    label_ids = b_labels.to('cpu').numpy()

    tmp_eval_accuracy = flat_accuracy(logits, label_ids)
    
    eval_accuracy += tmp_eval_accuracy
    nb_eval_steps += 1

  print("Validation Accuracy: {}".format(eval_accuracy/nb_eval_steps))

Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Train loss: 1.241741740463564


Epoch:  33%|███▎      | 1/3 [01:05<02:11, 65.54s/it]

Validation Accuracy: 0.4522058823529412
Train loss: 1.037006847410394


Epoch:  67%|██████▋   | 2/3 [02:07<01:03, 63.48s/it]

Validation Accuracy: 0.4338235294117647
Train loss: 0.9797991914237105


Epoch: 100%|██████████| 3/3 [03:09<00:00, 63.27s/it]

Validation Accuracy: 0.5845588235294118
